In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
#this is used to upload files into the browser
from google.colab import files
uploaded = files.upload()
path=next(iter(uploaded))

KeyboardInterrupt: 

In [ ]:
num_speakers = 2 #@param {type:"integer"} #how specify the number of speakers involved
#should specify the language below
language = 'any' #@param ['any','English']
#Here we should we specify at how much precise the txt must be. If we want more precise we need to add larger else we need to add small.
model_size = "small" #@param ['tiny','base','small','medium','large']
model_name=model_size
if language =='English' and model_size!='large':
  model_name+='.en'
  #This is done using agglomerative clustring where we need to specify the number of speakers so that it will work accordingly.

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
#We are installing whisper library and pyannote which will be used for seperating the diferent speakers in the Audio
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
if path[-3:] != 'wav' :
  #using ffmpeg it is converting into wav format
  subprocess.call(['ffmpeg','-i',path,'audio.wav','-y'])
  path = 'audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames/float(rate)

In [ ]:
audio = Audio()
def segment_embedding(segemnt):
  start = segment["start"]
  #Whisper overshoots the end timestamp in the last segment
  end = min(duration,segment["end"])
  clip = Segment(start,end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [ ]:
embeddings = np.zeros(shape=(len(segments),192))
for i,segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

  embedding = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = "SPEAKER" + str(labels[i]+1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt","w")

for (i,segment) in enumerate(segments):
  if i==0 or segments[i-1]["speaker"]!=segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' '+ str(time(segment["start"]))+'\n')
  f.write(segment["text"][1:]+' ')
f.close()


In [ ]:
print(open('transcript.txt' , 'r').read())


SPEAKER1 0:00:00
Do you have plans? 
SPEAKER2 0:00:01
As a matter of fact, they do. 
SPEAKER1 0:00:02
I don't like it when you have plans. 
SPEAKER2 0:00:04
I'm allowed to have plans on my birthday. 
SPEAKER1 0:00:06
It's your birthday? Yes. I knew that. Already? 
SPEAKER2 0:00:09
Yeah, isn't that strange? It's the same day as last year. 
SPEAKER1 0:00:13
Well, get yourself something nice for me. 
SPEAKER2 0:00:15
I already did. 
SPEAKER1 0:00:16
And? 
SPEAKER2 0:00:17
Oh, it was very nice. 
SPEAKER1 0:00:19
Very tasteful. Thank you, Mr. Stark. You're welcome as possible. 
